In [25]:
%load_ext autoreload
%autoreload 2
import torch
import numpy as np

from tp_pt_vae import models

n_tracks = 5
n_measures = 4
measure_resolution = 48
n_beats = 4
n_pitches = 84
d_latent = 64

vae = models.VAE(
    n_tracks,
    n_measures,
    measure_resolution,
    n_beats,
    n_pitches,
    d_latent,
)

_sample = torch.FloatTensor(np.array([np.load("/home/ksawada/Documents/lab/lab_research/musegan/data/extracted/data/71843.npy").astype(np.float32), ]))
_sample = torch.permute(_sample, (0, 4, 1, 2, 3))

out = vae(_sample)
print(out[0][0].shape)




The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
torch.Size([1, 5, 4, 48, 84])


In [11]:
from torch.utils.data import DataLoader
from tqdm import tqdm


from tp_pt_vae.datasets import PianorollDataset
from tp_pt_vae.bin.train import Collater

n_tracks = 5
n_measures = 4
measure_resolution = 48
n_beats = 4
n_pitches = 84
d_latent = 64
batch_size = 1
num_workers = 1
pin_memory = True

dataset = PianorollDataset(
    pianoroll_list="/home/ksawada/Documents/lab/lab_research/tp_pt_vae/egs/lpd/data/lpd_train.txt",
    n_tracks=n_tracks,
    measure_resolution=measure_resolution,
    n_pitches=n_pitches,
    n_measures=n_measures,
    allow_cache=False,
)
collater = Collater()

dataloader = DataLoader(
    dataset=dataset,
    shuffle=True,
    collate_fn=collater,
    batch_size=batch_size,
    num_workers=num_workers,
    sampler=None,
    pin_memory=pin_memory,
)

total = len(dataset) * n_tracks * n_measures * measure_resolution * n_pitches
ones = 0
for i in tqdm(dataloader):
    ones += (i == 0).sum().item()

print(f"ones/total = {ones} / {total} = {ones/total}")

 98%|█████████▊| 90505/92140 [32:30<00:05, 276.89it/s]Traceback (most recent call last):
  File "/home/ksawada/Documents/lab/lab_research/tp_pt_vae/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1466, in __del__
  File "/home/ksawada/Documents/lab/lab_research/tp_pt_vae/venv/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1449, in _shutdown_workers
  File "/home/ksawada/.pyenv/versions/3.9.1/lib/python3.9/multiprocessing/process.py", line 160, in is_alive
AssertionError: can only test a child process
100%|██████████| 92140/92140 [33:03<00:00, 46.44it/s] 

ones/total = 7289648693 / 7430169600 = 0.9810877928008535


In [12]:
1 - 0.9810877928008535

0.018912207199146547

In [5]:
import numpy as np

lpd = np.load("/home/ksawada/Documents/lab/lab_research/musegan/data/extracted/data/8938.npy")
print(lpd.shape)
lod_small = np.load("/home/ksawada/Documents/lab/lab_research/musegan-pytorch/data/all/00004109_TRJKCQE128F932623B/50.npy")
print(lod_small.shape)
print("batch: torch.Size([64, 5, 4, 48, 84])")

(4, 48, 84, 5)
(5, 64, 72)
batch: torch.Size([64, 5, 4, 48, 84])


# データセットのプレビュー

In [8]:
import os
import imageio
import numpy as np

colormap = np.array([
    [1., 0., 0.],
    [1., .5, 0.],
    [0., 1., 0.],
    [0., 0., 1.],
    [0., .5, 1.],
]).T

def pianoroll_to_image(pianoroll, colormap=None, inverted=True,
                       boundary_width=1, boundary_color=0, frame=False,
                       gamma=1.):
    """
    Convert a batched pianoroll array to an image array.

    Arguments
    ---------
    pianoroll : `np.array`, ndim=5
        The pianoroll array. The shape is (n_pianorolls, n_bars, n_timestep,
        n_pitches, n_tracks).
    boundary_width : int
        Linewidth of the boundary lines. Default to 0.
    boundary_color : int
        Grayscale of the boundary lines. Valid values are 0 (black) to 255
        (white). Default to 0.
    frame : bool
        Whether to use a grid frame. Default to False.

    Returns
    -------
    image : `np.array`, ndim=4
        The image array.
    """
    if pianoroll.ndim != 5:
        raise ValueError("Input pianoroll array must have 5 dimensions.")

    # Flip the pitch axis
    pianoroll = np.flip(pianoroll, 3)

    # Apply the color
    if colormap is not None:
        pianoroll = np.matmul(1. - colormap, np.expand_dims(pianoroll, -1))
        pianoroll = pianoroll.squeeze(-1).clip(0., 1.)

    # Apply gamma correction
    if gamma != 1.:
        pianoroll = pianoroll ** gamma

    # Invert the color
    if inverted:
        pianoroll = 1. - pianoroll

    # Quantize the image (minus a small value to avoid casting 256 to 0)
    quantized = (pianoroll * 256 - 1e-5).astype(np.uint8)

    # Add the boundary lines
    if boundary_width:
        quantized = np.pad(
            quantized,
            ((0, 0), (0, 0), (boundary_width, 0), (boundary_width, 0), (0, 0)),
            'constant', constant_values=boundary_color)

    # Transpose and reshape to get the image array
    if colormap is None:
        transposed = np.transpose(quantized, (0, 4, 3, 1, 2))
        image = np.reshape(
            transposed, (-1, transposed.shape[1] * transposed.shape[2],
                         transposed.shape[3] * transposed.shape[4], 1))
    else:
        transposed = np.transpose(quantized, (0, 3, 1, 2, 4))
        image = np.reshape(transposed, (
            -1, transposed.shape[1], transposed.shape[2] * transposed.shape[3],
            transposed.shape[4]))

    # Deal with the frame
    if boundary_width:
        if frame:
            image = np.pad(
                image,
                ((0, 0), (0, boundary_width), (0, boundary_width), (0, 0)),
                'constant', constant_values=boundary_color)
        else:
            image = image[:, boundary_width:, boundary_width:]

    return image

def image_grid(image, grid_shape, grid_width=3, grid_color=0, frame=True):
    """
    Convert a batched image array to one merged grid image array.

    Arguments
    ---------
    pianoroll : `np.array`, ndim=4
        The pianoroll array. The first axis is the batch axis. The second and
        third axes are the time and pitch axes, respectively, of the pianorolls.
        The last axis is the track axis.
    grid_shape : list or tuple of int
        Shape of the image grid (height, width).
    grid_width : int
        Linewidth of the grid. Default to 0.
    grid_color : int
        Grayscale of the grid. Valid values are 0 (black) to 255 (white).
        Default to 0.
    frame : bool
        Whether to use a grid frame. Default to False.

    Returns
    -------
    merged : `np.array`, ndim=3
        The merged image grid array.
    """
    if len(grid_shape) != 2:
        raise ValueError("`grid_shape` must be a list or tuple of two "
                         "integers.")
    if image.ndim != 4:
        raise ValueError("Input image array must have 4 dimensions.")

    # Slice the array to get the right number of images
    sliced = image[:(grid_shape[0] * grid_shape[1])]

    # Add the grid lines
    if grid_width:
        sliced = np.pad(
            sliced, ((0, 0), (grid_width, 0), (grid_width, 0), (0, 0)),
            'constant', constant_values=grid_color)

    # Reshape to split the first (batch) axis into two axes
    reshaped = np.reshape(sliced, ((grid_shape[0], grid_shape[1])
                                   + sliced.shape[1:]))

    # Transpose and reshape to get the image grid
    transposed = np.transpose(reshaped, (0, 2, 1, 3, 4))
    grid = np.reshape(
        transposed, (grid_shape[0] * transposed.shape[1],
                     grid_shape[1] * transposed.shape[3], image.shape[-1]))

    # Deal with the frame
    if grid_width:
        if frame:
            grid = np.pad(grid, ((0, grid_width), (0, grid_width), (0, 0)),
                          'constant', constant_values=grid_color)
        else:
            grid = grid[:, grid_width:, grid_width:]

    return grid

def _array_to_image(array, colormap=None):
    """Convert an array to an image array and return it."""
    # if array.ndim == 2:
    #     return vector_to_image(array)
    return pianoroll_to_image(array, colormap)

def _save_colored_images(array, suffix, name):
    """Save the input image."""
    if 'hard_thresholding' in name:
        array = (array > 0).astype(np.float32)
    elif 'bernoulli_sampling' in name:
        rand_num = np.random.uniform(size=array.shape)
        array = (.5 * (array + 1.) > rand_num)
        array = array.astype(np.float32)
    images = _array_to_image(array, colormap)
    return _save_image_grid(images, suffix, name)

def _get_filepath(folder_name, name, suffix, ext):
    """Return the filename."""
    if suffix:
        return os.path.join(
            "./", folder_name, name,
            '{}_{}.{}'.format(name, str(suffix, 'utf8'), ext))
    return os.path.join(
        "./", folder_name, name,
        '{}.{}'.format(name, ext))

def _save_image_grid(array, suffix, name):
    image = image_grid(array, [1, 1])
    filepath = _get_filepath('images', name, suffix, 'png')
    imageio.imwrite(filepath, image)
    return np.array([0], np.int32)

lpd = np.load("/home/ksawada/Documents/lab/lab_research/musegan/data/extracted/data/56628.npy")
print(lpd.shape)


_save_colored_images(np.array([lpd]), None, "test")




(4, 48, 84, 5)


array([0], dtype=int32)